Utilizando esse conjunto de dados:
https://www.kaggle.com/datasets/disham993/9000-movies-dataset

Responda as seguintes perguntas:

1.   Qual tamanho do DataSet?
2.   Quantas linhas?
3.   Quantas colunas?
4.   Qual o tipo de variável de cada coluna?
5.   Qual o filme com maior número de votaçôes?
6.   Qual filme teve a maior nota (critério de desempate é o filme com mais votos)
7.   Existem valores nulos? Se sim, qual tratamento irá realizar? (Se não temos nome de algum filme, melhor nem considerar)
8.   Transforme as variaveis categóricas de linguagem e genero em númericas (utilize dummy)
9.   Normalize as variaveis numéricas
10.  Armazene esses valores como um artefato dentro do MLFlow
11. QUais insights é possivel obter desses dados?



In [52]:
%pip install kagglehub seaborn numpy matplotlib

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [59]:
import kagglehub
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


# Bibliotecas para aprendizado de máquina
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, OrdinalEncoder

import mlflow


In [ ]:
# importing dataset



# Download latest version
dataset_path = kagglehub.dataset_download("disham993/9000-movies-dataset")


In [ ]:



df_movies = pd.read_csv(dataset_path + "/mymoviedb.csv", lineterminator='\n')

In [ ]:
df_movies.info()

# 1.   Qual tamanho do DataSet?
#   R: 691.1+ KB
# 2.   Quantas linhas?
#   R: 9827
# 3.   Quantas colunas?
#   R: 9
# 4.   Qual o tipo de variável de cada coluna?
#   R: float64(2), int64(1), object(6)


In [ ]:
# 5.   Qual o filme com maior número de votaçôes? R: Inception

id_max_count = df_movies.Vote_Count.idxmax()
df_movies.loc[id_max_count]


In [ ]:
# 6.   Qual filme teve a maior nota (critério de desempate é o filme com mais votos) R: Kung Fu Master Huo Yuanjia
df_movies.sort_values(by=['Vote_Average', 'Vote_Count'], ascending=[False, False]).iloc[0]


In [49]:
# 7.   Existem valores nulos? Se sim, qual tratamento irá realizar? (Se não temos nome de algum filme, melhor nem considerar)

df_movies.isnull().sum() + df_movies.isna().sum()


Release_Date         0
Title                0
Overview             0
Popularity           0
Vote_Count           0
Vote_Average         0
Original_Language    0
Genre                0
Poster_Url           0
dtype: int64

In [ ]:
# 8. Transforme as variaveis categóricas de linguagem e genero em númericas (utilize dummy)
df_copy = df_movies.copy()

# Diminui as quantidade de categorias
genre_counts = df_copy['Genre'].value_counts()
common_genres = genre_counts[genre_counts > 100].index
df_copy['Genre'] = df_copy['Genre'].where(df_copy['Genre'].isin(common_genres), 'Other')
lang_counts = df_copy['Original_Language'].value_counts()

common_langs = lang_counts[lang_counts > 50].index

df_copy['Original_Language'] = df_copy['Original_Language'].where(df_copy['Original_Language'].isin(common_langs), 'Other')
df_dummies = pd.get_dummies(df_copy, columns=['Genre', 'Original_Language'], dtype=int, drop_first=True)
df_dummies

,Release_Date,Title,Overview,Popularity,Vote_Count,Vote_Average,Poster_Url,Genre_Comedy,"Genre_Comedy, Drama","Genre_Comedy, Drama, Romance",...,Original_Language_cn,Original_Language_de,Original_Language_en,Original_Language_es,Original_Language_fr,Original_Language_it,Original_Language_ja,Original_Language_ko,Original_Language_ru,Original_Language_zh
0,2021-12-15,Spider-Man: No Way Home,Peter Parker is unmasked and no longer able to...,5083.954,8940,8.3,https://image.tmdb.org/t/p/original/1g0dhYtq4i...,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,2022-03-01,The Batman,"In his second year of fighting crime, Batman u...",3827.658,1151,8.1,https://image.tmdb.org/t/p/original/74xTEgt7R3...,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,2022-02-25,No Exit,Stranded at a rest stop in the mountains durin...,2618.087,122,6.3,https://image.tmdb.org/t/p/original/vDHsLnOWKl...,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,2021-11-24,Encanto,"The tale of an extraordinary family, the Madri...",2402.201,5076,7.7,https://image.tmdb.org/t/p/original/4j0PNHkMr5...,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,2021-12-22,The King's Man,As a collection of history's worst tyrants and...,1895.511,1793,7.0,https://image.tmdb.org/t/p/original/aq4Pwv5Xeu...,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9822,1973-10-15,Badlands,A dramatization of the Starkweather-Fugate kil...,13.357,896,7.6,https://image.tmdb.org/t/p/original/z81rBzHNgi...,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9823,2020-10-01,Violent Delights,A female vampire falls in love with a man she ...,13.356,8,3.5,https://image.tmdb.org/t/p/original/4b6HY7rud6...,0,0,0,...,0,0,0,1,0,0,0,0,0,0
9824,2016-05-06,The Offering,When young and successful reporter Jamie finds...,13.355,94,5.0,https://image.tmdb.org/t/p/original/h4uMM1wOhz...,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9825,2021-03-31,The United States vs. Billie Holiday,Billie Holiday spent much of her career being ...,13.354,152,6.7,https://image.tmdb.org/t/p/original/vEzkxuE2sJ...,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [83]:
# 9.   Normalize as variaveis numéricas

dados_scaled_standard = df_movies.copy()

num_cols = df_movies.select_dtypes(include=[np.number]).columns
standard = StandardScaler()
dados_scaled_standard[num_cols] = standard.fit_transform(dados_scaled_standard[num_cols])

print('Visualização após Padronização (Standard):')
display(dados_scaled_standard[num_cols].describe().T.head())

Visualização após Padronização (Standard):


,count,mean,std,min,25%,50%,75%,max
Popularity,9827.0,4.627530e-17,1.000051,-0.247749,-0.222264,-0.175690,-0.047163,46.327724
Vote_Count,9827.0,3.470647e-17,1.000051,-0.533422,-0.477507,-0.363377,-0.006436,11.368576
Vote_Average,9827.0,5.003517e-16,1.000051,-5.700208,-0.477590,0.053524,0.584638,3.151687


In [ ]:
# Prepara o dataset final

final_df_path = "final_dataset.csv"
final_df = df_dummies.copy()
final_df[num_cols] = dados_scaled_standard[num_cols] = standard.fit_transform(dados_scaled_standard[num_cols])

final_df.to_csv(final_df_path, index=False)
print("Dataset processado salvo localmente.")


  Release_Date                    Title  \
0   2021-12-15  Spider-Man: No Way Home   
1   2022-03-01               The Batman   
2   2022-02-25                  No Exit   
3   2021-11-24                  Encanto   
4   2021-12-22           The King's Man   

                                            Overview  Popularity  Vote_Count  \
0  Peter Parker is unmasked and no longer able to...   46.327724    2.890456   
1  In his second year of fighting crime, Batman u...   34.788147   -0.092608   
2  Stranded at a rest stop in the mountains durin...   23.677757   -0.486698   
3  The tale of an extraordinary family, the Madri...   21.694758    1.410605   
4  As a collection of history's worst tyrants and...   17.040610    0.153268   

   Vote_Average                                         Poster_Url  \
0      1.646865  https://image.tmdb.org/t/p/original/1g0dhYtq4i...   
1      1.469827  https://image.tmdb.org/t/p/original/74xTEgt7R3...   
2     -0.123514  https://image.tmdb.org/t/p/origin

In [96]:
# Insights 11. QUais insights é possivel obter desses dados?

# Entende distribuição geral: quais filmes têm maior destaque e se há outliers.
print(final_df[['Popularity', 'Vote_Count', 'Vote_Average']].describe())

# Pode revelar se filmes populares também têm notas altas (ou não).
print(final_df[['Popularity', 'Vote_Count', 'Vote_Average']].corr())

# Lista os mais populares.
print(final_df.nlargest(10, 'Popularity')[['Title', 'Popularity', 'Vote_Average']])

final_df['Year'] = pd.to_datetime(final_df['Release_Date']).dt.year

#final_df['Year'].value_counts().sort_index().plot(kind='bar')
#final_df.groupby('Year')['Popularity'].mean().plot()

#Correlacao de genero e nota
genre_cols = [c for c in final_df.columns if c.startswith('Genre_')]
lang_cols = [c for c in final_df.columns if c.startswith('Original_Language_')]
final_df.groupby(lang_cols)[genre_cols].mean()
print(final_df[genre_cols + ['Vote_Average']].corr()['Vote_Average'].sort_values(ascending=False))







        Popularity   Vote_Count  Vote_Average
count  9827.000000  9827.000000  9.827000e+03
mean      0.000000     0.000000  2.892206e-18
std       1.000051     1.000051  1.000051e+00
min      -0.247749    -0.533422 -5.700208e+00
25%      -0.222264    -0.477507 -4.775898e-01
50%      -0.175690    -0.363377  5.352392e-02
75%      -0.047163    -0.006436  5.846376e-01
max      46.327724    11.368576  3.151687e+00
              Popularity  Vote_Count  Vote_Average
Popularity      1.000000    0.137400      0.053844
Vote_Count      0.137400    1.000000      0.253574
Vote_Average    0.053844    0.253574      1.000000
                     Title  Popularity  Vote_Average
0  Spider-Man: No Way Home   46.327724      1.646865
1               The Batman   34.788147      1.469827
2                  No Exit   23.677757     -0.123514
3                  Encanto   21.694758      1.115751
4           The King's Man   17.040610      0.496119
5             The Commando   15.708479      0.142043
6          

In [ ]:
# 10.  Armazene esses valores como um artefato dentro do MLFlow

mlflow.start_run()

mlflow.log_artifact(final_df_path)  # Registrar o arquivo como artefato

mlflow.end_run()

print("Features armazenadas e versionadas com sucesso no MLflow!")


Features armazenadas e versionadas com sucesso no MLflow!
